In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gtsa
import pandas as pd

### Lazy load co-registered DEMs from S3

In [ ]:
s3_bucket_name = 'petrichor'
folder = 'baker/dems/'

In [ ]:
cog_urls = gtsa.io.parse_urls_from_S3_bucket(s3_bucket_name,
                                             aws_server_url = 's3.us-west-2.amazonaws.com',
                                             folder = folder)

date_strings = gtsa.io.parse_timestamps(cog_urls,
                                        date_string_pattern='....-..-..')

date_times = [pd.to_datetime(x) for x in date_strings]

In [ ]:
len(cog_urls) == len(date_times)

In [ ]:
ref_dem = cog_urls[-1]
ref_dem

In [ ]:
ds = gtsa.io.xr_stack_geotifs(cog_urls,
                              date_times,
                              ref_dem,
                              resampling="bilinear",
                              save_to_nc = True,
                              nc_out_dir = '/mnt/working/knuth/sw/repos/gtsa/scripts/stacks/baker/nc_files',
                              overwrite = False)

In [ ]:
ds

In [ ]:
ds['band1'].data = ds['band1'].data.rechunk({0:'auto', 1:'auto', 2:'auto'}, 
                                                    block_size_limit=1e8, 
                                                    balance=True)

In [ ]:
ds

In [ ]:
ds['band1'].sel(time = ds.time.values[0])

### Write to Zarr file

In [ ]:
ds_zarr = gtsa.io.create_zarr_stack(ds,
                                    output_directory = './zarr_stack_rainier',
                                    overwrite = False)

In [ ]:
ds_zarr

In [ ]:
ds_zarr['band1'].sel(time = ds_zarr.time.values[0])